In [31]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName('Practice').getOrCreate()
path = "C:\\Users\\ARNAB ADHIKARY\\sample.json"
df2 = spark.read.option("multiLine", "true").option("mode", "PERMISSIVE").option("inferSchema", "true").json(path)
df2.printSchema()

root
 |-- datasets: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- customerId: string (nullable = true)
 |    |    |-- orderDate: string (nullable = true)
 |    |    |-- orderDetails: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- productId: string (nullable = true)
 |    |    |    |    |-- quantity: long (nullable = true)
 |    |    |    |    |-- sequence: long (nullable = true)
 |    |    |    |    |-- totalPrice: struct (nullable = true)
 |    |    |    |    |    |-- gross: long (nullable = true)
 |    |    |    |    |    |-- net: long (nullable = true)
 |    |    |    |    |    |-- tax: long (nullable = true)
 |    |    |-- orderId: string (nullable = true)
 |    |    |-- shipmentDetails: struct (nullable = true)
 |    |    |    |-- city: string (nullable = true)
 |    |    |    |-- country: string (nullable = true)
 |    |    |    |-- postalCode: string (nullable = true)
 |    |    

In [32]:
from pyspark.sql import types as T
import pyspark.sql.functions as F


def flatten(df):
    complex_fields = dict([
        (field.name, field.dataType) 
        for field in df.schema.fields 
        if isinstance(field.dataType, T.ArrayType) or isinstance(field.dataType, T.StructType)
    ])
    
    qualify = list(complex_fields.keys())[0] + "_"

    while len(complex_fields) != 0:
        col_name = list(complex_fields.keys())[0]
        
        if isinstance(complex_fields[col_name], T.StructType):
            expanded = [F.col(col_name + '.' + k).alias(col_name + '_' + k) 
                        for k in [ n.name for n in  complex_fields[col_name]]
                       ]
            
            df = df.select("*", *expanded).drop(col_name)
    
        elif isinstance(complex_fields[col_name], T.ArrayType): 
            df = df.withColumn(col_name, F.explode(col_name))
    
      
        complex_fields = dict([
            (field.name, field.dataType)
            for field in df.schema.fields
            if isinstance(field.dataType, T.ArrayType) or isinstance(field.dataType, T.StructType)
        ])
        
        
    for df_col_name in df.columns:
        df = df.withColumnRenamed(df_col_name, df_col_name.replace(qualify, ""))

    return df

In [33]:
flatten(df2)

DataFrame[filename: string, customerId: string, orderDate: string, orderId: string, orderDetails_productId: string, orderDetails_quantity: bigint, orderDetails_sequence: bigint, shipmentDetails_city: string, shipmentDetails_country: string, shipmentDetails_postalCode: string, shipmentDetails_state: string, shipmentDetails_street: string, orderDetails_totalPrice_gross: bigint, orderDetails_totalPrice_net: bigint, orderDetails_totalPrice_tax: bigint]

In [36]:
print(df2)
df2.write.format("delta").mode("overwrite").save("C:\\Users\\ARNAB ADHIKARY")

DataFrame[datasets: array<struct<customerId:string,orderDate:string,orderDetails:array<struct<productId:string,quantity:bigint,sequence:bigint,totalPrice:struct<gross:bigint,net:bigint,tax:bigint>>>,orderId:string,shipmentDetails:struct<city:string,country:string,postalCode:string,state:string,street:string>>>, filename: string]


Py4JJavaError: An error occurred while calling o337.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:863)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:257)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:240)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:75)
	at java.base

In [47]:
df2.write.format("parquet").mode("overwrite").partitionBy("quantity").save("sample_parquet")

AnalysisException: Partition column `quantity` not found in schema struct<datasets:array<struct<customerId:string,orderDate:string,orderDetails:array<struct<productId:string,quantity:bigint,sequence:bigint,totalPrice:struct<gross:bigint,net:bigint,tax:bigint>>>,orderId:string,shipmentDetails:struct<city:string,country:string,postalCode:string,state:string,street:string>>>,filename:string>.

In [49]:
flights = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("2008.csv")

In [53]:
flights.write.format("parquet") \
.mode("overwrite") \
.partitionBy("Origin") \
.save("flights_parquet")

Py4JJavaError: An error occurred while calling o420.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java